# 💘 The Stolen Valentine Budget
## A Corporate SQL Investigation - Introduction

Lunaris Systems approved a modest Valentine’s Day budget for its Madrid office.

The plan was simple:
a small internal celebration to thank employees for their work.

The budget was approved. Expense reports were submitted. Everything appeared routine.

Until Finance reviewed the February numbers.

The Valentine budget appears to have been quietly drained.

Several expense reports contain purchases that raise questions.
Audit suspects that one employee may have disguised personal spending as legitimate office expenses.

Your mission:

> Use SQL to investigate the data and identify the prime suspect.

This will not be a step-by-step recipe.

You will need to:

* Explore the company’s expense data
* Test investigative hypotheses
* Distinguish real clues from misleading ones
* Apply precise filtering and aggregation
* Produce a final, defensible SQL report

If your reasoning is correct, your final query will return **exactly one employee**.

If it isn’t… you may accuse the wrong person.


## **SQL Environment Setup (do not edit)**

In [1]:
# @title
%%capture
!mkdir -p notebook_lib
!wget -q -O notebook_lib/sql_runner.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/8021f5c05b7d973b8db549a1398a3c9a5c7829d5/notebook_lib/sql_runner.py
!wget -q -O notebook_lib/validators.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/7baff2c6485cdf641cabcdb55d92a51317cd18b9/notebook_lib/validators.py

from notebook_lib.sql_runner import make_sql_runner
from notebook_lib.validators import make_df_validator_nospoilers, check_process_rules

import sqlite3
import pandas as pd
from pathlib import Path


In [2]:
# @title
DB_FILE = 'class.db'

if DB_FILE != ":memory:":
    Path(DB_FILE).unlink(missing_ok=True)

conn = sqlite3.connect(DB_FILE)
conn.execute("PRAGMA foreign_keys = ON;")

conn.executescript(r"""

-- 💘 The Stolen Valentine Budget — Sample Dataset (SQLite)
-- Create + Insert script
-- Designed to yield EXACTLY 1 correct prime suspect under the intended rules.

PRAGMA foreign_keys = ON;

-- ----------------------------
-- Drop tables (safe re-run)
-- ----------------------------
DROP TABLE IF EXISTS expenses;
DROP TABLE IF EXISTS expense_reports;
DROP TABLE IF EXISTS valentine_budget;
DROP TABLE IF EXISTS vendors;
DROP TABLE IF EXISTS employees;

-- ----------------------------
-- 1) employees
-- ----------------------------
CREATE TABLE employees (
  employee_id INTEGER PRIMARY KEY,
  first_name  TEXT NOT NULL,
  last_name   TEXT NOT NULL,
  department  TEXT NOT NULL,
  office      TEXT NOT NULL
);

-- ----------------------------
-- 2) vendors
-- ----------------------------
CREATE TABLE vendors (
  vendor_id    INTEGER PRIMARY KEY,
  vendor_name  TEXT NOT NULL,
  vendor_type  TEXT NOT NULL
);

-- ----------------------------
-- 3) valentine_budget
-- ----------------------------
CREATE TABLE valentine_budget (
  office          TEXT NOT NULL,
  budget_month    TEXT NOT NULL,   -- YYYY-MM
  approved_budget REAL NOT NULL,
  PRIMARY KEY (office, budget_month)
);

-- ----------------------------
-- 4) expense_reports
-- ----------------------------
CREATE TABLE expense_reports (
  report_id       INTEGER PRIMARY KEY,
  employee_id     INTEGER NOT NULL,
  submitted_date  TEXT NOT NULL,   -- YYYY-MM-DD
  status          TEXT NOT NULL,   -- Approved / Pending / Rejected
  FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);

-- ----------------------------
-- 5) expenses
-- ----------------------------
CREATE TABLE expenses (
  expense_id    INTEGER PRIMARY KEY,
  report_id     INTEGER NOT NULL,
  vendor_id     INTEGER NOT NULL,
  expense_date  TEXT NOT NULL,     -- YYYY-MM-DD
  category      TEXT NOT NULL,     -- Meals / Gifts / Flowers / Office / ...
  amount        REAL NOT NULL,
  receipt_id    TEXT,              -- NULL allowed
  FOREIGN KEY (report_id) REFERENCES expense_reports(report_id),
  FOREIGN KEY (vendor_id) REFERENCES vendors(vendor_id)
);

-- ============================================================
-- INSERTS
-- ============================================================

-- valentine budget (flavor / exploration bait)
INSERT INTO valentine_budget (office, budget_month, approved_budget) VALUES
('Madrid',   '2026-02', 600.0);

-- employees
INSERT INTO employees (employee_id, first_name, last_name, department, office) VALUES
(1, 'Eva',   'Martín',  'Sales',     'Madrid'),
(2, 'Luis',  'Ortega',  'HR',        'Madrid'),
(3, 'Carla', 'Núñez',   'Marketing', 'Madrid'),
(4, 'Tom',   'Ríos',    'Finance',   'Madrid'),
(5, 'Sara',  'Vidal',   'Sales',     'Barcelona');  -- trap (wrong office)

-- vendors
INSERT INTO vendors (vendor_id, vendor_name, vendor_type) VALUES
(10, 'Amor & Flores', 'Florist'),
(11, 'La Cena',       'Restaurant'),
(12, 'Regalos Lux',   'Gift Shop'),
(13, 'OfficeMaxi',    'Office Supplies'),
(14, 'ChocoCasa',     'Gift Shop');

-- expense_reports
INSERT INTO expense_reports (report_id, employee_id, submitted_date, status) VALUES
(100, 1, '2026-02-16', 'Approved'),
(101, 1, '2026-02-20', 'Approved'),
(102, 2, '2026-02-18', 'Approved'),
(103, 3, '2026-02-15', 'Approved'),
(104, 4, '2026-02-19', 'Approved'),
(105, 3, '2026-02-21', 'Pending'),   -- trap
(106, 2, '2026-02-22', 'Rejected'),  -- trap
(107, 5, '2026-02-17', 'Approved');  -- trap (wrong office)

-- expenses
-- Eva (employee_id=1) — intended PRIME SUSPECT under correct rules
INSERT INTO expenses (expense_id, report_id, vendor_id, expense_date, category, amount, receipt_id) VALUES
(2000, 100, 11, '2026-02-12', 'Meals',  220.0, 'R-2000'),
(2001, 100, 12, '2026-02-12', 'Gifts',  180.0, 'R-2001'),
(2002, 101, 12, '2026-02-14', 'Gifts',  250.0, 'R-2002'),
(2003, 101, 14, '2026-02-18', 'Gifts',  160.0, 'R-2003'),

-- Extra date trap: Eva has a big Gifts expense on Jan 31 (should be excluded by Feb filter)
(2050, 100, 12, '2026-01-31', 'Gifts',  700.0, 'R-2050');

-- Carla (employee_id=3) — traps: big Office purchase + Pending huge Gift + Flowers bait
INSERT INTO expenses (expense_id, report_id, vendor_id, expense_date, category, amount, receipt_id) VALUES
(2010, 103, 13, '2026-02-10', 'Office',  520.0, 'R-2010'),  -- NOT Meals/Gifts
(2011, 103, 11, '2026-02-14', 'Meals',    40.0, 'R-2011'),  -- small legit
(2012, 105, 12, '2026-02-13', 'Gifts',   600.0, 'R-2012'),  -- PENDING report (should be excluded)
(2013, 103, 10, '2026-02-14', 'Flowers',  90.0, 'R-2013');  -- Flowers bait

-- Luis (employee_id=2) — meets count>=3 but fails distinct_vendors>=2 (all same vendor)
INSERT INTO expenses (expense_id, report_id, vendor_id, expense_date, category, amount, receipt_id) VALUES
(2020, 102, 11, '2026-02-11', 'Meals', 110.0, 'R-2020'),
(2021, 102, 11, '2026-02-14', 'Meals', 120.0, 'R-2021'),
(2022, 102, 11, '2026-02-17', 'Meals', 130.0, 'R-2022');

-- Luis rejected trap (should be excluded via report status)
INSERT INTO expenses (expense_id, report_id, vendor_id, expense_date, category, amount, receipt_id) VALUES
(2060, 106, 12, '2026-02-20', 'Gifts', 400.0, 'R-2060');  -- REJECTED report

-- Tom (employee_id=4) — traps: huge amounts but missing receipts
INSERT INTO expenses (expense_id, report_id, vendor_id, expense_date, category, amount, receipt_id) VALUES
(2030, 104, 12, '2026-02-14', 'Gifts', 500.0, NULL),
(2031, 104, 11, '2026-02-16', 'Meals', 320.0, NULL),
(2032, 104, 11, '2026-02-18', 'Meals',  60.0, 'R-2032');

-- Sara (employee_id=5, Barcelona) — trap: would win if office filter is missing
INSERT INTO expenses (expense_id, report_id, vendor_id, expense_date, category, amount, receipt_id) VALUES
(2040, 107, 12, '2026-02-14', 'Gifts', 900.0, 'R-2040'),
(2041, 107, 11, '2026-02-15', 'Meals', 200.0, 'R-2041'),
(2042, 107, 14, '2026-02-16', 'Gifts', 150.0, 'R-2042');

-- ============================================================
-- Optional sanity checks (for you)
-- ============================================================

-- Quick peek:
-- SELECT * FROM employees;
-- SELECT * FROM expense_reports;
-- SELECT * FROM expenses;

-- If you want to see who the correct suspect SHOULD be (based on the intended rules),
-- your final query should return exactly Eva (employee_id = 1) as top row.



""")
print(f"Database ready ✅ ({DB_FILE})")


Database ready ✅ (class.db)


## 🗂️ The Database

The investigation involves five tables:

1.  employees: Information about Lunaris employees.
2.  vendors: Businesses where purchases were made.
3.  expense_reports: Expense reports submitted by employees.
4. expenses: Individual expense line items inside each report.
5. valentine_budget: The officially approved Valentine budget for February.

You will need to combine data from multiple tables to solve this case.

In [15]:
# @title ER Diagram
%%html
<img id="er-img" style="width:80%; max-width:100%; height:auto;"
     data-light="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/ER_stolen_valentine_budget.png"
     data-dark="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/ER_stolen_valentine_budget_black.png"
     alt="ER diagram">

<script>
  const img = document.getElementById("er-img");

  function isDarkTheme() {
    // Colab sets html[theme=dark] on the top document
    const themeAttr = document.documentElement.getAttribute("theme");
    if (themeAttr) return themeAttr === "dark";

    // fallback: OS/browser preference
    return window.matchMedia && window.matchMedia("(prefers-color-scheme: dark)").matches;
  }

  function updateImage() {
    img.src = isDarkTheme() ? img.dataset.dark : img.dataset.light;
  }

  updateImage();

  // React to Colab theme toggles (attribute changes)
  new MutationObserver(updateImage).observe(document.documentElement, {
    attributes: true,
    attributeFilter: ["theme"]
  });

  // React to OS/browser theme changes (fallback)
  if (window.matchMedia) {
    const mq = window.matchMedia("(prefers-color-scheme: dark)");
    mq.addEventListener?.("change", updateImage);
    mq.addListener?.(updateImage); // older browsers
  }
</script>

## ⚖️ Investigation Rules

You may use only the SQL concepts covered so far:

* `SELECT`, `FROM`, `WHERE`
* `DISTINCT`
* `AND`, `OR`
* `ORDER BY`
* `JOIN ... ON`
* Aggregate functions (`COUNT`, `SUM`, `AVG`, `MIN`, `MAX`)
* `GROUP BY`
* `HAVING`

You may **not** use:

* Advanced SQL features not covered in class

The objective is to solve the case using clean relational reasoning.

## 🎯 Final Objective

By the end of this notebook, you must write **one SQL query** that:

* Applies the correct investigative filters
* Aggregates suspicious expenses per employee
* Returns exactly one row — the prime suspect
* Sorts results correctly so the suspect appears at the top

If your logic is sound, the suspect will be unambiguous.

Welcome to your first corporate investigation.


## 📘 How This Investigation Is Structured

This notebook is divided into three sections:

### Section 1 — Evidence Room

You will explore the database and understand how the tables relate to each other.

Some questions in this section require you to compute values using SQL and submit the **result** on Canvas, which are marked with:

>  📝 Submit on Canvas

These require you to compute the answer using SQL and submit the **result** on Canvas.

All other exploration queries are optional but strongly recommended.

### Section 2 — Hypothesis Testing

You will test investigative hypotheses and analyze patterns in the data.

Some questions will also be marked:

> 📝 Submit on Canvas

These require a specific answer or short explanation on Canvas.

Other questions are exploratory and intended to guide your reasoning toward the final investigation.

### Section 3 — Discover the Prime Suspect

You will write one final SQL query that identifies the prime suspect.

This is the most important part of the assignment.

This final query must be submitted on Canvas.



# 🔎 Section 1 — The Evidence Room (20%)

Before drawing conclusions, investigators examine the evidence.

In this section, your goal is simple:

> Understand the structure of the data and what information is available.

Do not try to identify the suspect yet.
First, learn how the expense system works.

In [3]:
# @title 1.1 - The Valentine Budget

make_sql_runner(
    conn,
    runner_id="ex1_1",
    description_md="""
## 🧾 1.1 The Valentine Budget

Let’s begin with the official budget.

What budget was approved for the Madrid office in February 2026?

Run a query to display the contents of the `valentine_budget` table.

Take note of:

* The office
* The month
* The approved amount

This establishes the scope of the investigation.

""",
    )



In [4]:
# @title 1.2 - Who Works at Lunaris?

make_sql_runner(
    conn,
    runner_id="ex1_2",
    description_md="""
## 👥 1.2. Who Works at Lunaris?

Which employees are part of the company?
Which office are they assigned to?

Display all information from the `employees` table.

Questions to consider:

* How many offices are represented? 📝 **Submit on Canvas**
* Are all employees located in Madrid?
* Will office location matter later?
""",
    )



In [5]:
# @title 1.3 - Where Are Purchases Made?

make_sql_runner(
    conn,
    runner_id="ex1_3",
    description_md="""
## 🏢 3. Where Are Purchases Made?

Expenses are made through external vendors.

Explore the `vendors` table.

Then check the different types of vendors available.

Think:

* What kinds of purchases might be considered suspicious?
* Which vendor types might be normal for office use?

Do not draw conclusions yet, just observe.

""",
    )



In [6]:
# @title 1.4 - How Do Expense Reports Work?

make_sql_runner(
    conn,
    runner_id="ex1_4",
    description_md="""
## 🧾 1.4 How Do Expense Reports Work?

Explore the `expense_reports` table.
Expense reports group multiple expense line items.

Questions to consider:

* How many different report statuses exist in the expense_reports table? 📝 **Submit on Canvas**
* Do all submitted reports actually affect the company budget?

""",
    )



In [7]:
# @title 1.5 - What Do Individual Expenses Look Like?

make_sql_runner(
    conn,
    runner_id="ex1_5",
    description_md="""
## 💳 1.5 What Do Individual Expenses Look Like?

The real details live in the `expenses` table. Explore it and look at:

* Distinct categories:
* Date range:
* Missing receipts (null):

Observe carefully. Some details may become important later.

Question:

* What is the earliest expense_date in the expenses table? 📝 Submit on Canvas
* How many expenses have a missing receipt (receipt_id IS NULL)? 📝 Submit on Canvas
""",
    )



## End of Section 1

At this stage, you should understand:

* How the five tables relate to each other
* What information each table contains
* Which columns may be relevant for filtering
* Which values look potentially meaningful

You are not solving the case yet.

You are building context.

# 🧠 Section 2 — Testing the Hypotheses (30%)

Finance and Audit provided several possible leads.

Not all of them are reliable.

Your job in this section is to **test each hypothesis using SQL**.

Some clues will turn out to be useful.
Others may be misleading.

As investigators, you must rely on evidence, not intuition.


In [8]:
# @title Hypothesis 1 — Only Approved Reports Affect the Budget

make_sql_runner(
    conn,
    runner_id="hypothesis_1",
    description_md="""
## 🧾 Hypothesis 1: Only Approved Reports Affect the Budget

Audit suggests that only **approved expense reports** actually impacted Lunaris’ budget.

Before accepting this claim, investigate how reports are distributed across different statuses.

### 🔎 Suggested exploration

To evaluate this hypothesis, consider examining:

* How many reports exist per `status`
* Whether multiple status values appear in the system
* Whether all statuses logically represent expenses that affect the company budget

Use SQL to explore these questions.

Think carefully:

* Should Pending or Rejected reports be treated the same way as Approved?
* If an expense “affects the budget,” does that mean: The company has already paid it? Or that it is still under review?
* Which report statuses represent expenses that have actually been reimbursed?

Do not decide yet. Just observe patterns in the data.

""",
    )



In [9]:
# @title Hypothesis 2 — The Theft Happened on Valentine’s Day (February 14)

make_sql_runner(
    conn,
    runner_id="hypothesis_2",
    description_md="""
## 💘 Hypothesis 2: The Theft Happened on Valentine’s Day (February 14)

A rumor suggests the suspicious activity occurred exactly on February 14.

Before trusting that claim, examine whether spending is concentrated on a single day or distributed across the month.

### 🔎 Suggested exploration

To evaluate this hypothesis, consider examining:

* The full range of dates in the `expenses` table
* How many expenses occur per day in February
* Whether total spending peaks on a specific date

Avoid filtering prematurely to February 14.

Let the data speak first.

""",
    )



In [10]:
# @title 📝 Submit This Query — Spending by Date (Daily totals)

make_sql_runner(
    conn,
    runner_id="hypothesis_submit_2",
    description_md="""
## Is spending concentrated on a single day?

Write a SQL query that returns, for each day:

* `expense_date`
* number of expenses (`COUNT(*)` as `expenses_count`)
* total spending (`SUM(amount)` as `total_amount`)

Apply the following conditions:

* Office = **Madrid**
* `expense_date` in **February 2026**
* report `status` = **'Approved'**

Your result must:

* Group by `expense_date`
* Order by `expense_date` (ascending)

Do **not** filter to a single day.
""",
    )



In [11]:
# @title Hypothesis 3 — Certain Categories Are Suspicious

make_sql_runner(
    conn,
    runner_id="hypothesis_3",
    description_md="""
## 🌹 Hypothesis 3 — Certain Categories Are Suspicious

Audit believes the theft may be disguised under specific expense categories. However, they are unsure which categories show unusual spending.

To evaluate this hypothesis, consider:

* Total spending per `category`
* Which categories dominate February spending
* Whether some categories look consistent with business activities
* Whether others appear more personal or Valentine-related

Focus on:

* Office = Madrid
* February 2026
* Approved reports only

You may want to rank categories by total spending.

After analyzing your results, reflect:

* Which categories appear unusually high?
* Which seem legitimate?
* Which might warrant closer scrutiny?

You will use this reasoning later.
""",
    )



In [12]:
# @title Hypothesis 4 — Certain Categories Are Suspicious

make_sql_runner(
    conn,
    runner_id="hypothesis_4",
    description_md="""
## 🧾 Hypothesis 4: Missing Receipts May Indicate Suspicion

Audit flagged that some expenses lack receipt documentation.

Missing receipts can indicate weak documentation or potentially something more serious.

### 🔎 Suggested exploration

To evaluate this hypothesis, consider:

* How many expenses have `receipt_id IS NULL`
* Whether missing receipts appear randomly
* Whether they cluster around specific vendors

Pay attention to:

* Office = Madrid
* February 2026
* Approved reports only

""",
    )



In [13]:
# @title 📝 Submit This Query — Missing Receipts by Vendor

make_sql_runner(
    conn,
    runner_id="hypothesis_submit_3",
    description_md="""
## Do missing receipts cluster around specific vendors?

Write a SQL query that returns:

* `vendor_name`
* number of expenses with missing receipts (`COUNT(*)` as `missing_receipts_count`)
* total amount for expenses with missing receipts (`SUM(amount)` as `missing_receipts_total_amount`)

Only count expenses where:

* `receipt_id IS NULL`

Apply the following conditions:

* Office = **Madrid**
* `expense_date` in **February 2026**
* report `status` = **'Approved'**

Your result must:

* Group by `vendor_name`
* Order by `missing_receipts_count` (descending), then by `missing_receipts_total_amount` (descending)

Do **not** apply any additional filters.
""",
    )



At this stage, you should have:

* Examined how report statuses affect interpretation
* Determined whether February 14 alone explains the spending pattern
* Identified which categories dominate spending
* Investigated whether missing receipts cluster around specific vendors

Now comes the critical step:

> Which filters are logically necessary to identify the true suspect?

The final investigation will require precision.
Over-filtering may eliminate the suspect.
Under-filtering may accuse the wrong person.

Proceed carefully.

# ⚖️ Section 3 — The Final Case Report

You have tested the hypotheses.

You have examined:

* Report statuses
* Daily spending patterns
* Category-level totals
* Receipt irregularities

Now Audit requires a formal conclusion.


## Audit Request

Audit requires a clear and defensible SQL report identifying the prime suspect in the Valentine budget case.

Your task:

> Write one SQL query that identifies the employee whose February spending patterns indicate misuse of the Valentine budget.

You must determine:

* Which filters are logically necessary
* Which clues were misleading
* Which expense categories are relevant
* Which records should be excluded
* Which aggregation rules meaningfully identify suspicious behavior

Your query must rely only on data-driven logic. Do not hard-code names or values unrelated to the investigation.

If more than one employee appears, your logic is incomplete.
If no employee appears, your filtering is too restrictive.
If the wrong employee appears, your reasoning was flawed.

In [14]:
# @title 3.1 Discover the Prime Suspect

make_sql_runner(
    conn,
    runner_id="3_1",
    description_md="""
## 🔍 3.1 Discover the Prime Suspect

Your final query must return exactly these columns:

* `employee_id`
* `first_name`
* `last_name`
* `suspicious_expenses_count`
* `total_suspicious_amount`
* `distinct_vendors`
* `avg_expense_amount`

Sort your results so the most suspicious employee appears first.

Only one name should remain.

This is the report that will be sent to senior management.

Choose your filters carefully.

""",
    )

